In [1]:
import pandas as pd
from transformers import pipeline
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pd.read_csv('/content/drive/MyDrive/IUT/IR/project/4. embedding & language models/train.csv')

sampled_data = data.sample(n=1000, random_state=42)
sentences = sampled_data['Title'].tolist()

In [3]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import pandas as pd

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def generate_sentence(sentence, mask="[MASK]"):
    tokens = tokenizer.tokenize(sentence)

    masked_index = len(tokens) // 2
    tokens[masked_index] = mask

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_ids_tensor = torch.tensor([input_ids])

    with torch.no_grad():
        outputs = model(input_ids_tensor)
        predictions = outputs.logits

    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    tokens[masked_index] = predicted_token

    return tokenizer.convert_tokens_to_string(tokens)

augmented_sentences = []

for sentence in sentences:
    new_sentence = generate_sentence(sentence)
    augmented_sentences.append(new_sentence)

augmented_data = pd.DataFrame({
    'Original': sentences,
    'Generated': augmented_sentences
})

augmented_data.to_csv('/content/drive/MyDrive/IUT/IR/project/4. embedding & language models/augmented_dataset.csv', index=False)

print("Dataset has been saved as 'augmented_dataset.csv'")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

Dataset has been saved as 'augmented_dataset.csv'
